In [202]:
fact= {
'O' : "Outside", 
'B-geo' : "Geographical Entity",
'B-gpe' : "Geopolitical Entity", 
'B-per' : "PERSON",
'I-geo' : "Geographical Entity",                  # B- Begining 
'B-org' : "ORGANIZATION",                         # I - Inside
'I-org' : "ORGANIZATION", 
'B-tim' : "TIME",
'B-art' : "Artifact", 
'I-art' : "Artifact", 
'I-per' : "PERSON", 
'I-gpe' : "Geopolitical Entity", 
'I-tim' : "TIME", 
'B-nat' : "Natural Phenomenon", 
'B-eve' : "Event",
'I-eve' : "Event",
'I-nat' : "Natural Phenomenon"
 }

In [114]:
import nltk
import random
import pandas as pd
from pathlib import Path
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# SPACY

In [346]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NLP/data_ner.csv")
data = data.iloc[:-1]
data = data.fillna(method="ffill", axis=0)

In [347]:
LABEL = list(data["Tag"].unique())

In [349]:
data = data.groupby(["Sentence #"], as_index=False
             )["Word", "POS", "Tag"].agg(lambda x: list(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [350]:
def create_data(index):
  
  position = 0
  a = data["Tag"].iloc[index]
  temp = []

  for j in range(len(a)):
    
    if a[j]!="O":

      temp.append((position, position + len(data["Word"].iloc[j]) , a[j]))
    position += len(data["Word"].iloc[j])
  return temp

# creating datatset according to the spacy requirements

dataset = []
for i in range(data.shape[0]):

  content = " ".join(data.iloc[i]["Word"])
  entities = create_data(i)

  dataset.append(
            (
                content, {
                    "entities" : entities
                }
            )
  )

In [351]:
# Setting up the pipeline and entity recognizer.
model = None
if model is not None:
    nlp = spacy.load(model)  # load existing spacy model
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  # create blank Language class
    print("Created blank 'en' model")
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner)
else:
    ner = nlp.get_pipe('ner')

Created blank 'en' model


In [352]:
for i in LABEL:
    ner.add_label(i)
# Inititalizing optimizer
if model is None:
    optimizer = nlp.begin_training()
else:
    optimizer = nlp.entity.create_optimizer()

In [353]:
from spacy.util import minibatch, compounding

In [374]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

TRAIN_DATA = dataset
n_iter = 10

with nlp.disable_pipes(*other_pipes):  # only train NER
    for itn in range(n_iter):
        print("Iteration ",itn," #"*100)
        random.shuffle(TRAIN_DATA)
        losses = {}
        batches = minibatch(TRAIN_DATA, 
                            size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch) 
            # Updating the weights
            nlp.update(texts, annotations, sgd=optimizer, 
                       drop=0.35, losses=losses)
            print('Losses', losses)
            nlp.update(texts, annotations, sgd=optimizer, 
                       drop=0.35, losses=losses)
            print('Losses', losses)

In [359]:
output_dir = "/content/drive/MyDrive/Colab Notebooks/NLP"
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.meta['name'] = "spacy_101_ner"  # rename model
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Saved model to /content/drive/MyDrive/Colab Notebooks/NLP


In [371]:
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
doc2 = nlp2("""Prime Minister Jacinda Ardern has claimed that New Zealand had won a big 
battle over the spread of coronavirus. Her words came as the country begins to exit from its lockdown.""")
displacy.render(doc2, style = 'ent', jupyter=True)

Loading from /content/drive/MyDrive/Colab Notebooks/NLP
